## 패키지 설치

In [1]:
# [설명] Python 3.12 확인
import sys
print(sys.version)


3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [2]:
# [설명]
# 3.12에서 충돌이 잦은 bitsandbytes/triton 제외
# RAG 필수 패키지만 설치
!pip -q install -U --no-cache-dir \
  numpy \
  transformers \
  tokenizers \
  accelerate \
  sentence-transformers \
  chromadb \
  rank-bm25 \
  scikit-learn \
  langsmith


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 276.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 360.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 345.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 384.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 399.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 411.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 395.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 409.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 409.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 398.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 416.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 116.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored in directory: /root/.cache/pip/wheels/3d/59/46/f282c12c73dd4bb3c2e3fe199f1a0d0f8cec06df0cccfeee27
Successfully built flash-attn


In [4]:
# [설명] DB 경로 접근
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


LangSmith 설정

In [5]:
# [설명] traceable이 자동으로 LangSmith에 기록되도록 설정
import os
from google.colab import userdata

api_key = userdata.get("langgraph")
assert api_key, "Colab Secret likelion-langsmith 없음"

os.environ["LANGCHAIN_TRACING_V2"] = "False"
os.environ["LANGCHAIN_API_KEY"] = api_key
os.environ["LANGCHAIN_PROJECT"] = "likelion-medichat"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

print("LangSmith ready")


LangSmith ready


In [6]:
!pip -q uninstall -y numpy
!pip -q uninstall -y numpy
!pip -q install -q --upgrade --force-reinstall --no-cache-dir "numpy==2.0.2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 125.1 MB/s eta 0:00:00


In [7]:
# !pip -q install -U --no-cache-dir \
#   "transformers" \
#   "tokenizers" \
#   "accelerate" \
#   "sentence-transformers>=3.3.0" \
#   "chromadb" \
#   "rank-bm25" \
#   "scikit-learn" \
#   "langsmith"


## 디바이스 + 임베딩 + reranker 로드

In [8]:
# [설명] bge-m3 임베딩(1024) + CrossEncoder reranker
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

emb_model = SentenceTransformer("BAAI/bge-m3", device=device)

def embed_texts(texts):
    return emb_model.encode(texts, normalize_embeddings=True, show_progress_bar=False)

reranker = CrossEncoder("BAAI/bge-reranker-v2-m3", device=device)
print("embed + rerank ready")


device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

embed + rerank ready


## Chroma 로드 + BM25 코퍼스 구성

In [9]:
# [설명] chroma-re 컬렉션 로드 + 문서 덤프 + BM25 인덱스 생성
import chromadb
from rank_bm25 import BM25Okapi

DB_PATH = "/content/drive/MyDrive/med_chatbot/chroma_db"
COLLECTION_NAME = "med_knowledge"

client = chromadb.PersistentClient(path=DB_PATH)
col = client.get_collection(COLLECTION_NAME)

print("doc count:", col.count())

dump = col.get(include=["documents","metadatas"])
all_docs = dump["documents"]
all_metas = dump["metadatas"]
print("loaded docs:", len(all_docs))

def simple_tokenize(s):
    if not isinstance(s, str):
        return []
    return s.replace("\n", " ").split()

bm25 = BM25Okapi([simple_tokenize(t) for t in all_docs])
print("BM25 ready")


doc count: 184340
loaded docs: 184340
BM25 ready


## EXAONE fp16 로딩

In [10]:
# [설명]
# bitsandbytes 없이 fp16 로딩 (3.12 안전)
# A100 아니면 느릴 수 있음
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_ID = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    trust_remote_code=True
)

gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=False,
    temperature=0,
    batch_size=32
)

print("EXAONE bf16 ready")


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

...

model-00002-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


EXAONE bf16 ready


## 프롬프트 정의

In [11]:
# [설명] q_type별 프롬프트 생성기 (chat template 지원)
SYSTEM_PROMPT = """너는 의료 정보 안내용 챗봇이다.
확정적인 진단을 내리지 말고, 일반적인 의학 정보를 바탕으로 설명한다.
근거가 불충분하면 추측하지 말고 모른다고 말한다.
위험 신호가 보이면 반드시 의료기관 방문을 권한다.
"""

def build_prompt(tokenizer, question: str, q_type: int):
    if q_type == 1:
        user_content = f"""
다음은 객관식 의료 문제이다.

반드시 정답에 해당하는 선택지를
"번호) 선택지 내용" 형식으로만 답하라.
질문과 선택지를 다 읽고 가장 적절한 답변을 간결하게 답하라.
다른 설명은 절대 하지 말라.

질문:
{question.strip()}
""".strip()

    elif q_type == 2:
        user_content = f"""
다음 질문에 대해 단답으로 답하라.

규칙:
단어 또는 짧은 구절로만 답하라.
불필요한 설명을 하지 말라.
근거가 불충분하면 "모르겠습니다"라고만 답하라.

질문:
{question.strip()}
""".strip()

    else:
        user_content = f"""
다음 질문에 대해 일반적인 의학 정보를 바탕으로 설명하라.

규칙:
핵심 내용에 대해 한 문장으로 간결히 답하라.
불필요한 설명을 하지 말라.
근거가 불충분하면 "모르겠습니다"라고만 답하라.

질문:
{question.strip()}
""".strip()

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content},
    ]

    if hasattr(tokenizer, "apply_chat_template"):
        return tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

    return f"{SYSTEM_PROMPT}\n{user_content}\n답변:"


## 전체 파이프라인 (LangSmith 추적)

In [12]:
# [설명]
# vector → bm25 → merge → rerank → generate 전체 단계 traceable로 추적
import numpy as np
from langsmith import traceable

def get_id(text, meta):
    m = meta or {}
    return m.get("file_id") or m.get("file_name") or text[:80]

@traceable(name="retrieve_vector")
def retrieve_vector(query, fetch_k=40):
    q_emb = embed_texts([query])[0].tolist()
    res = col.query(
        query_embeddings=[q_emb],
        n_results=fetch_k,
        include=["documents","metadatas","distances"]
    )
    docs = res["documents"][0]
    metas = res["metadatas"][0]
    dists = res["distances"][0]
    out = []
    for t, m, d in zip(docs, metas, dists):
        out.append({"text": t, "meta": m or {}, "score": float(-d)})
    return out

@traceable(name="retrieve_bm25")
def retrieve_bm25(query, k=30):
    scores = bm25.get_scores(simple_tokenize(query))
    idx = np.argsort(scores)[::-1][:k]
    out = []
    for i in idx:
        out.append({"text": all_docs[i], "meta": all_metas[i] or {}, "score": float(scores[i])})
    return out

@traceable(name="merge_candidates")
def merge_candidates(vec_items, bm_items, max_items=80):
    merged = {}
    for it in bm_items:
        merged[get_id(it["text"], it["meta"])] = it
    for it in vec_items:
        key = get_id(it["text"], it["meta"])
        if key not in merged:
            merged[key] = it
    return list(merged.values())[:max_items]

@traceable(name="rerank")
def rerank(query, items, top_k=5):
    if not items:
        return []
    pairs = [(query, it["text"]) for it in items]
    scores = reranker.predict(pairs)
    scored = list(zip(items, scores))
    scored.sort(key=lambda x: x[1], reverse=True)
    return [it for it, _ in scored[:top_k]]

def build_context(ctx_items):
    parts = []
    for i, it in enumerate(ctx_items, 1):
        m = it.get("meta", {}) or {}
        fid = m.get("file_id") or m.get("file_name") or "unknown"
        parts.append(f"[{i}] {fid}\n{it['text']}")
    return "\n\n".join(parts)

@traceable(name="generate")
def generate_answer(question, q_type, ctx_items):
    ctx = build_context(ctx_items)

    base = build_prompt(tokenizer, question, int(q_type))

    prompt = (
        f"{base}\n\n"
        f"[참고자료]\n{ctx}\n\n"
        f"답변:"
    )

    out = gen(prompt)[0]["generated_text"]
    return out[len(prompt):].strip()

@traceable(name="rag_pipeline")
def rag_answer(question, q_type, mode="hybrid_rerank"):
    if mode == "vector":
        vec = retrieve_vector(question, fetch_k=20)
        ctx_items = vec[:5]

    elif mode == "bm25":
        ctx_items = retrieve_bm25(question, k=5)

    elif mode == "hybrid":
        vec = retrieve_vector(question, fetch_k=20)[:10]
        bm = retrieve_bm25(question, k=20)
        merged = merge_candidates(vec, bm, max_items=40)
        ctx_items = merged[:5]

    elif mode == "hybrid_rerank":
        vec = retrieve_vector(question, fetch_k=20)
        bm = retrieve_bm25(question, k=20)
        merged = merge_candidates(vec, bm, max_items=40)
        ctx_items = rerank(question, merged, top_k=5)

    else:
        raise ValueError("unknown mode")

    ans = generate_answer(question, q_type, ctx_items)
    return ans, ctx_items


테스트

In [13]:
# [설명] end-to-end 실행 + LangSmith에서 trace 확인
q = "레보노르게스트렐이 포함된 응급 피임약의 주요 작용 기전은 무엇인가?"
ans, items = rag_answer(q, q_type=3, mode="hybrid_rerank")
print(ans[:1200])
print("ctx n:", len(items))


레보노르게스트렐이 포함된 응급 피임약의 주요 작용 기전은 배란 억제와 자궁내막의 변화를 통해 수정란의 착상을 방지하는 것입니다.
ctx n: 5


## 새로운 평가

In [14]:
!pip install -U bert_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00


In [15]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00


In [16]:
import pandas as pd
import numpy as np
import json
import re
import os
import ast
from tqdm import tqdm
import evaluate
from datasets import Dataset

# 데이터 로드
EVAL_ITEMS_PATH = "/content/drive/MyDrive/med_chatbot/eval_items.jsonl"
OUT_CSV_PATH = "/content/drive/MyDrive/med_chatbot/eval_gold_pred.csv"

items = []
print(f"Loading data from {EVAL_ITEMS_PATH}...")

if not os.path.exists(EVAL_ITEMS_PATH):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {EVAL_ITEMS_PATH}")

with open(EVAL_ITEMS_PATH, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        line = line.strip()
        if not line: continue
        try:
            item = json.loads(line)
            if "question" not in item: continue
            items.append(item)
        except json.JSONDecodeError:
            continue

print(f"Total items loaded: {len(items)}")

# 이어하기 로직
processed_ids = set()

if os.path.exists(OUT_CSV_PATH):
    try:
        existing_df = pd.read_csv(OUT_CSV_PATH)
        if "qa_id" in existing_df.columns:
            processed_ids = set(existing_df["qa_id"].astype(str))
            print(f"기존 진행 상황 감지: {len(processed_ids)}개 완료됨. 이어서 진행합니다.")
    except Exception as e:
        print(f"기존 파일 읽기 실패 (새로 시작): {e}")

items_to_process = [x for x in items if str(x.get("qa_id")) not in processed_ids]
print(f"남은 처리 대상: {len(items_to_process)}개")

# 예측 함수
def predict_one_with_context(ex):
    q = ex["question"]
    try:
        q_type = int(ex.get("q_type", 3))
    except:
        q_type = 3

    ans, ctx_items = rag_answer(q, q_type=q_type, mode="hybrid_rerank")

    context_texts = [item['text'] for item in ctx_items]

    return (ans or "").strip(), context_texts

# 메인 평가 루프
batch_rows = []
SAVE_INTERVAL = 100

print("Starting Evaluation Loop on A100...")

for i, ex in enumerate(tqdm(items_to_process, desc="Generating Answers")):
    try:
        pred, contexts = predict_one_with_context(ex)
        gold = ex.get("answer", "")

        batch_rows.append({
            "qa_id": ex.get("qa_id"),
            "domain": ex.get("domain"),
            "q_type": int(ex.get("q_type", 3)),
            "split": ex.get("split"),
            "label": ex.get("label"),
            "question": ex.get("question"),
            "gold_answer": gold,
            "pred_answer": pred,
            "contexts": contexts
        })

        if len(batch_rows) >= SAVE_INTERVAL or i == len(items_to_process) - 1:
            df_batch = pd.DataFrame(batch_rows)

            if not os.path.exists(OUT_CSV_PATH):
                df_batch.to_csv(OUT_CSV_PATH, index=False, encoding="utf-8-sig", mode='w')
            else:
                df_batch.to_csv(OUT_CSV_PATH, index=False, encoding="utf-8-sig", mode='a', header=False)

            batch_rows = []

    except Exception as e:
        print(f"\n[Error processing item]: {ex.get('qa_id', 'unknown')} - {e}")
        continue

print(f"\nAll predictions completed and saved to {OUT_CSV_PATH}")

# 전체 데이터 로드 및 변환
df_final = pd.read_csv(OUT_CSV_PATH)
print(f"Loaded full dataset for metrics: {len(df_final)} rows")

def safe_eval_list(x):
    if isinstance(x, str) and x.startswith("["):
        try:
            return ast.literal_eval(x)
        except:
            return []
    return x if isinstance(x, list) else []

df_final["contexts"] = df_final["contexts"].apply(safe_eval_list)

# 객관식 평가
def extract_choice(text):
    if not isinstance(text, str): return None
    t = text.replace("\r", "\n").strip()
    patterns = [r"\b([1-5])\s*\)", r"\b([1-5])\s*번\b", r"\b([1-5])\b"]
    for p in patterns:
        m = re.findall(p, t)
        if m: return m[-1]
    return None

if not df_final.empty:
    mc = df_final[df_final["q_type"] == 1].copy()
    if not mc.empty:
        mc["gold_choice"] = mc["gold_answer"].apply(extract_choice)
        mc["pred_choice"] = mc["pred_answer"].apply(extract_choice)

        mc_valid = mc.dropna(subset=["gold_choice", "pred_choice"])
        acc = float((mc_valid["gold_choice"] == mc_valid["pred_choice"]).mean()) if len(mc_valid) else 0.0

        print("-" * 30)
        print(f"[Type 1] 객관식 평가 결과")
        print(f"Total: {len(mc)}, Valid: {len(mc_valid)}")
        print(f"Accuracy: {acc:.4f}")
        print("-" * 30)

# BERTScore 계산
print("[Type 2,3] BERTScore 계산 중...")

bertscore = evaluate.load("bertscore")

def compute_bertscore_f1(golds, preds, batch_size=64):
    out = bertscore.compute(
        predictions=[str(x) for x in preds],
        references=[str(x) for x in golds],
        model_type="xlm-roberta-large",
        lang="ko",
        batch_size=batch_size,
        device="cuda"
    )
    return np.array(out["f1"], dtype=float)

mask = df_final["q_type"].isin([2, 3])
sub = df_final.loc[mask].copy()

if len(sub) > 0:
    f1_scores = compute_bertscore_f1(
        sub["gold_answer"].fillna(""),
        sub["pred_answer"].fillna(""),
        batch_size=64
    )

    df_final["bertscore_f1"] = np.nan
    df_final.loc[mask, "bertscore_f1"] = f1_scores

    for qt in [2, 3]:
        qt_vals = df_final.loc[df_final["q_type"] == qt, "bertscore_f1"].dropna()
        mean_val = qt_vals.mean() if len(qt_vals) > 0 else 0.0
        print(f"[Type {qt}] Mean F1: {mean_val:.4f} (n={len(qt_vals)})")

OUT_FINAL_CSV = "/content/drive/MyDrive/med_chatbot/eval_gold_pred_with_bertscore.csv"
df_final.to_csv(OUT_FINAL_CSV, index=False, encoding="utf-8-sig")
print(f"Final CSV saved: {OUT_FINAL_CSV}")

# RAGAS 데이터셋 저장
ragas_df = df_final[["question", "pred_answer", "contexts", "gold_answer", "q_type"]].copy()
ragas_df.columns = ["question", "answer", "contexts", "ground_truth", "q_type"]

RAGAS_JSON_PATH = "/content/drive/MyDrive/med_chatbot/ragas_eval_dataset.json"
ragas_df.to_json(RAGAS_JSON_PATH, orient="records", force_ascii=False)

print("-" * 30)
print(f"RAGAS 평가용 데이터셋 저장 완료: {RAGAS_JSON_PATH}")

Loading data from /content/drive/MyDrive/med_chatbot/eval_items.jsonl...
Total items loaded: 1000
기존 진행 상황 감지: 300개 완료됨. 이어서 진행합니다.
남은 처리 대상: 700개
Starting Evaluation Loop on A100...


Generating Answers: 100%|██████████| 700/700 [1:17:41<00:00,  6.66s/it]



All predictions completed and saved to /content/drive/MyDrive/med_chatbot/eval_gold_pred.csv
Loaded full dataset for metrics: 1000 rows
------------------------------
[Type 1] 객관식 평가 결과
Total: 731, Valid: 729
Accuracy: 0.6831
------------------------------
[Type 2,3] BERTScore 계산 중...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

[Type 2] Mean F1: 0.8808 (n=116)
[Type 3] Mean F1: 0.8859 (n=153)
Final CSV saved: /content/drive/MyDrive/med_chatbot/eval_gold_pred_with_bertscore.csv
------------------------------
RAGAS 평가용 데이터셋 저장 완료: /content/drive/MyDrive/med_chatbot/ragas_eval_dataset.json


In [1]:
# [설명] DB 경로 접근
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
!pip -q install -U "ragas" "langchain-core" "langchain-openai" "langchain-community"

Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.78
Uninstalling langchain-core-0.3.78:
  Successfully uninstalled langchain-core-0.3.78
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.5/466.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.2/490.2 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import userdata
from datasets import load_dataset
from ragas import evaluate
from ragas.metrics import (
    Faithfulness,
    ContextPrecision
)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 1. API Key 로드
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('openai')
except Exception as e:
    print("Secrets 설정을 확인하세요.")
    raise e

# 모델 load
judge_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    # 아래 옵션이 ```json 같은 마크다운 장식을 없애고 순수 JSON만 반환하게 만듭니다.
    model_kwargs={"response_format": {"type": "json_object"}}
)

judge_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 3. 데이터 로드
dataset = load_dataset("json", data_files="/content/drive/MyDrive/med_chatbot/ragas_eval_dataset.json", split="train")

# 4. Target 지표 2개만 설정
context_precision = ContextPrecision(llm=judge_llm) # 검색 품질 평가
faithfulness = Faithfulness(llm=judge_llm)          # 답변 신뢰성(할루시네이션) 평가

# 5. 평가 실행
print("Evaluating targeted metrics (Context Precision & Faithfulness)...")
results = evaluate(
    dataset,
    metrics=[
        context_precision,
        faithfulness
    ],
    raise_exceptions=False
)

print(results)

# 6. 결과 저장
save_path = "/content/drive/MyDrive/med_chatbot/ragas_results.csv"
results.to_pandas().to_csv(save_path, index=False)
print(f"Saved results to: {save_path}")

/tmp/ipython-input-3939960237.py:5: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Faithfulness
  from ragas.metrics import (
/tmp/ipython-input-3939960237.py:5: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import (


Evaluating targeted metrics (Context Precision & Faithfulness)...


Evaluating:   0%|          | 0/2000 [00:00<?, ?it/s]